# Comparaisons entre différentes manières de prédiction de notes

Ce notebook présente la MSE et la MAE pour différentes façons de prédire les notes d'utilisateurs sur des séries.  
4000 utilisateurs pour 3000 séries.  
Au moins 4 notes pour chacun des utilisateurs.  
Les différents algorithmes comparés :
- kSVD
- NMF
    - sans biais
    - avec biais
- content


In [19]:
from utils.recommandation_errors import *
import pickle

In [13]:
path_ratings = "/Vrac/PLDAC_addic7ed/ratings/ratings_imdb/users"
#path_ratings = "/Users/constancescherer/Desktop/ratings/ratings_imdb/users"
path_d_user = "/Vrac/PLDAC_addic7ed/pickles/d_user.p"
#path_d_user = "/Users/constancescherer/Desktop/pickles/d_user.p"
#d_user= get_d_user(path)
with open(path_d_user, 'rb') as pickle_file:
    d_user = pickle.load(pickle_file)

liste_series = get_liste_series(d_user)
data = get_data(d_user)
all_data, num_user, num_item = get_all_data(data)
train, train_mat, test = get_train_test(num_user, num_item, all_data, test_size=20)
mean, u_means, i_means,U_ksvd, I_ksvd =  get_Uksvd_Iksvd(train, train_mat, num_user, num_item)
d_username_id, d_itemname_id, Full = create_sparse_mat(data)

#path_series = "/Users/constancescherer/Desktop/addic7ed_final"
path_series = '/Vrac/PLDAC_addic7ed/addic7ed_final'
d_info, d_name = getDicts(path_series)
d_ind = reverse_dict(d_name)
d_titre_filename = get_d_titre_filename("titles/title-filename.txt")
d_filename_titre = reverse_dict(d_titre_filename)
d_id_username = reverse_dict(d_username_id)
d_id_serie = reverse_dict(d_itemname_id)

path_sim = "/Vrac/PLDAC_addic7ed/pickles/sim.p"
#path_sim = "/Users/constancescherer/Desktop/pickles/sim.p"
# matrice des similarités cosinus
with open(path_sim, 'rb') as pickle_file:
    similarities = pickle.load(pickle_file)

## Mean Only

In [13]:
error_mean_only(train_mat, test)

Training Error:
MSE: 8.105694
MAE: 2.3358402
Test Error:
MSE: 8.277842351522526
MAE: 2.373684267889131


## kSVD

In [7]:
error_ksvd(train_mat, test)

Training Error:
MSE: 1.4717542445892757
MAE: 0.7410327440155239
Test Error:
MSE: 5.641115231844463
MAE: 1.716289295576649


## NMF

### Sans biais

In [8]:
error_NMF(train_mat, test, num_user, num_item)

Training Error:
MSE: 0.0029776867
MAE: 0.0029776867
Test Error:
MSE: 8.944893460690668
MAE: 2.3056576047024246


In [3]:
error_NMF(train_mat, test, num_user, num_item)

Training Error:
MSE: 0.0020495765
MAE: 0.0020495765
Test Error:
MSE: 8.84276267450404
MAE: 2.2858192505510653


### Avec biais

In [9]:
error_NMF_biais(train_mat,test, 100, num_user, num_item)

/users/nfs/Etu6/3520166/PLDAC_Recommandation_analyse_sous_titres-master/utils/collaborative.py:180: RuntimeWarning: Mean of empty slice.
  item_mean = np.array(list(item.values())).mean()
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Training Error:
MSE: 2.0061487296492517
MAE: 0.6923314900034804
Test Error:
MSE: 6.0911094783247615
MAE: 1.6869948567229978


## Content

In [3]:
error_content(train_mat, test, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, similarities)

Training Error:
MSE: 3.6745684932389757
MAE: 1.3577477860706137
Test Error:
MSE: 3.6009470160829453
MAE: 1.3543962772471223


In [11]:
def error_content_return(train_mat, test, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, similarities) :
    """returns train and test error for content based recommandation with user/item biases using MSE and MAE metrics"""
    truth_tr = np.array([rating for (uid,iid),rating in train_mat.items()])
    truth_te = np.array([rating for uid,iid,rating in test])

    prediction_tr = np.array([pred_content(u, i, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, similarities) for (u,i),rating in train_mat.items()])
    prediction_te = np.array([pred_content(u, i, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, similarities) for u,i,rating in test])


    return MSE_err(prediction_tr,truth_tr), MAE_err(prediction_tr,truth_tr), MSE_err(prediction_te,truth_te), MAE_err(prediction_te,truth_te)

In [20]:
mse_train_er = []
mae_train_er = []
mse_test_er = []
mae_test_er = []
for size in range(100000, 600000, 100000):
    similarities = pickle.load(open("/Vrac/PLDAC_reco/pickles/sim/sim_voc_"+str(size)+".p", "rb"))
    mse_train, mae_train, mse_test, mae_test = error_content_return(train_mat, test, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, similarities)
    mse_train_er.append(mse_train)
    mae_train_er.append(mae_train)
    mse_test_er.append(mse_test)
    mae_test_er.append(mae_test)

In [21]:
mae_test_er

[1.3639480773940729,
 1.3622336517266715,
 1.3614988978692137,
 1.3627234876316432,
 1.3639480773940729]

La taille du vocabulaire n'influe pas trop

kSVD et content donnent de bons résultats en test.

La NMF sans biais est mauvaise en test (sur apprentissage) mais elle ne prend pas en compte les biais des utilisateurs et des items (contrairement au kSVD).
La NMF avec biais est meilleur que le kSVD.
Les prédictions des notes par rapport au contenu sont les meilleures.